In [1]:
import math
import pathlib
import pydpf
import torch
import model
import neural_networks
import training
import time
import pandas as pd
import datetime


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Prepare data

In [2]:
#prepare_data(device)

In [3]:
observation_encoding_size = 128
state_encoding_size = 64
scaling = 1000.
DPF_types = ['DPF', 'Soft', 'Stop-Gradient', 'Marginal Stop-Gradient', 'Optimal Transport', 'Kernel']
deterministic = True
if deterministic:
    results_file = 'deep_mind_maze_results.csv'
else:
    results_file = 'nondeterministic_deep_mind_maze_results.csv'

In [4]:
def flatten_gens(list_of_gens):
    return [item for gen in list_of_gens for item in gen]
    
def is_in_it(item, it):
    return any(id(item) == id(item_) for item_ in it)
    

def get_SSM():
    encoder = neural_networks.ObservationEncoder(observation_encoding_size, generator=cuda_gen, dropout_keep_ratio=0.3)
    decoder = neural_networks.ObservationDecoder(observation_encoding_size, generator=cuda_gen, dropout_keep_ratio=0.3)
    state_encoder = neural_networks.StateEncoder(state_encoding_size, generator=cuda_gen, dropout_keep_ratio=0.6)
    observation_partial_flows = [neural_networks.RealNVP_cond(dim = observation_encoding_size, hidden_dim=observation_encoding_size, condition_on_dim=state_encoding_size, generator = cuda_gen, zero_i=True), neural_networks.RealNVP_cond(dim = observation_encoding_size, hidden_dim=observation_encoding_size, condition_on_dim=state_encoding_size, generator = cuda_gen, zero_i=True)]
    flow_cov = torch.nn.Parameter(torch.eye(observation_encoding_size, device=device)*1, requires_grad=False)
    observation_flow = neural_networks.NormalizingFlowModel_cond(pydpf.MultivariateGaussian(torch.zeros(observation_encoding_size, device=device), cholesky_covariance= flow_cov, diagonal_cov=True, generator=cuda_gen), observation_partial_flows, device)
    observation_model = model.MazeObservation(observation_flow, encoder, decoder, state_encoder, device=device)
    dynamic_cov = torch.diag(torch.tensor([30/(scaling), 30/(scaling), 0.1], device=device))
    dynamic_model = model.MazeDynamic(cuda_gen, dynamic_cov)
    #proposal_partial_flows = [neural_networks.RealNVP_cond(dim = 3, hidden_dim=32, condition_on_dim=observation_encoding_size, generator=cuda_gen, zero_i=True), neural_networks.RealNVP_cond(dim = 3, hidden_dim=32, condition_on_dim=observation_encoding_size, generator=cuda_gen, zero_i=True)]
    #proposal_flow = neural_networks.NormalizingFlowModel_cond(None, proposal_partial_flows, device)
    #proposal_model = model.MazeProposal(proposal_flow, dynamic_model)
    prior_model = model.MazePrior(2*1000/scaling, 1.3*1000/scaling, cuda_gen)
    encoder_parameters = flatten_gens([encoder.parameters(), state_encoder.parameters(), decoder.parameters()])
    #flow_parameters = flatten_gens([observation_flow.parameters(), proposal_flow.parameters(), prior_model.parameters()])
    flow_parameters = flatten_gens([observation_flow.parameters(), prior_model.parameters()])
    SSM = pydpf.FilteringModel(dynamic_model=dynamic_model, prior_model=prior_model, observation_model=observation_model)
    print(f'observation encoder ps {sum(p.numel() for p in encoder.parameters())}')
    print(f'state encoder ps {sum(p.numel() for p in state_encoder.parameters())}')
    print(f'decoder ps {sum(p.numel() for p in decoder.parameters())}')
    print(f'observation flow ps {sum(p.numel() for p in observation_flow.parameters())}')
    #print(f'proposal flow ps {sum(p.numel() for p in proposal_flow.parameters())}')
    return SSM, encoder_parameters, flow_parameters, [flow_cov]
            

In [5]:
def transform_control(control, **data):
    output = control/torch.tensor([[[scaling, scaling, 1.]]], device=device)
    return output
    

In [6]:
def normalise_obs(observation, **data):
    return (observation - torch.mean(observation))/torch.std(observation)
    

In [7]:
def get_DPF(SSM):
    if DPF_type == 'DPF':
        return pydpf.DPF(SSM=SSM, resampling_generator=cuda_gen)
    if DPF_type == 'Soft':
        return pydpf.SoftDPF(SSM=SSM, resampling_generator=cuda_gen)
    if DPF_type == 'Stop-Gradient':
        return pydpf.StopGradientDPF(SSM=SSM, resampling_generator=cuda_gen)
    if DPF_type == 'Marginal Stop-Gradient':
        return pydpf.MarginalStopGradientDPF(SSM=SSM, resampling_generator=cuda_gen)
    if DPF_type == 'Optimal Transport':
        return pydpf.OptimalTransportDPF(SSM=SSM, regularisation=1.)
    if DPF_type == 'Kernel':
        Gaussian_kernel = pydpf.StandardGaussian(3, cuda_gen, learn_mean=False, learn_cov=True)
        kernel_mixture = pydpf.KernelMixture(kernel=Gaussian_kernel, generator=cuda_gen)
        return pydpf.KernelDPF(SSM=SSM, kernel=kernel_mixture)
    raise ValueError('DPF_type should be one of the allowed options')

In [8]:
with pydpf.utils.set_deterministic_mode(deterministic, True):
    for DPF_type in DPF_types:
        total_MSE = 0
        total_time = 0
        for i in range(5):
            cuda_gen = torch.Generator(device=device).manual_seed(i*10)
            SSM, encoder_params, flow_params, flow_cov = get_SSM()
            dpf = get_DPF(SSM)
            dpf.to(device)
            if DPF_type == 'Kernel':
                opt = torch.optim.AdamW([{'params': encoder_params, 'lr': 0.005}, {'params': flow_params, 'lr': 0.001}, {'params': dpf.resampler.mixture.parameters(), 'lr': 0.001, 'weight_decay': 0}], weight_decay=1e-3, betas=(0.7, 0.98), eps=1e-9)
            else:
                opt = torch.optim.AdamW([{'params': encoder_params, 'lr': 0.005}, {'params': flow_params, 'lr': 0.001}], weight_decay=1e-2, betas=(0.8, 0.99), eps=1e-9)
            opt_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.99)
            data = pydpf.StateSpaceDataset(data_path= pathlib.Path('.').parent.absolute().joinpath('data/maze_data.csv'), state_prefix='state', control_prefix='control', device = device)
            data.apply(normalise_obs,  'observation')
            scaling_tesnor = torch.tensor([[[scaling, scaling, 1.]]], device=device)
            data.apply(lambda state, **data: (state - torch.tensor([[[1000., 650., 0.]]], device=device))/scaling_tesnor, 'state')
            data.apply(transform_control, 'control')
            print('Data Loaded')
            start_time = time.time()
            test_mse, _ = training.train(dpf, opt, data, 100, (100, 100, 100), (64, 64, 64), (0.45, 0.2, 0.35), (1., 1., 1.), torch.Generator().manual_seed(i*10), None, 'MSE', 99, pre_train_epochs=0, device=device, lr_scheduler=opt_scheduler, state_scaling = scaling)
            total_MSE += test_mse
            total_time += time.time() - start_time
        MSE = total_MSE / 5
        runtime = total_time/5
        result_path = pathlib.Path('.').parent.absolute().joinpath(results_file)
        results = pd.read_csv(result_path, index_col=0)
        row = [str(datetime.timedelta(seconds=runtime)), math.sqrt(MSE)]
        results.loc[DPF_type] = row
        print(results)
        results.to_csv(result_path)


observation encoder ps 115808
state encoder ps 6896
decoder ps 116134
observation flow ps 181504


Deterministic mode enabled, this will increase runtime and memory costs and should only be used when reproduciblity is required, reproducibility is only guarenteed on a static hardware and software setup.


Data Loaded


cumsum_cuda_kernel does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\Context.cpp:95.)


epoch 1/100, train loss: 7.584640312194824, validation position RMSE: 677.7304497364715, validation angle RMSE: 1.819739415409054
epoch 2/100, train loss: 2.6231635983784996, validation position RMSE: 670.3418446956746, validation angle RMSE: 1.8205122285561584
epoch 3/100, train loss: 2.583584691153632, validation position RMSE: 643.110160664563, validation angle RMSE: 1.82195239417969
epoch 4/100, train loss: 2.554122808244493, validation position RMSE: 629.4575392748902, validation angle RMSE: 1.8236709118454744
epoch 5/100, train loss: 2.5294470426771376, validation position RMSE: 636.5867065844212, validation angle RMSE: 1.8219235238733984
epoch 6/100, train loss: 2.5070559734768336, validation position RMSE: 631.8015629926853, validation angle RMSE: 1.814645941172874
epoch 7/100, train loss: 2.491701987584432, validation position RMSE: 654.8919643727505, validation angle RMSE: 1.8142404894454018
epoch 8/100, train loss: 2.474055591159397, validation position RMSE: 680.40828000546

KeyboardInterrupt: 